# Lab 2: Markov Blanket in the IID Scenario

This notebook demonstrates:
1. Loading generated datasets with true DAG structure
2. Visualizing the true causal graph
3. Introduce three feature selection methods and corresponding training receipes:
    - Full feature set
    - Causal discovery based Markov Blanket search (CD-MB)
    - L1 loss
    - Oracle Markov Blanket (Oracle-MB)
4. Comparing results

In [ ]:
%load_ext watermark
%watermark -a "Muhammed Hunaid Topiwala" -v

%load_ext autoreload
%autoreload 2

In [ ]:
import logging
logging.disable(logging.WARNING)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datasets import load_dataset

from blanket.feature_selection import (
    direct_lingam_selector,
    ges_selector,
    notears_selector,
    pc_selector,
)
from blanket.models import (
    linear_l1_regression,
    linear_regression,
)

from blanket.metrics import adjacency_confusion, jaccard_score, reduction_rate, shd
from blanket.plots import plot_adjmat, plot_graph

## 1. Load Dataset and Select Example

In [ ]:
# Load dataset
train = load_dataset(path="CSE472-blanket-challenge/phase1-dataset", split='train', name='linear')

train.features

In [ ]:
train_example = train[70]

X = np.asarray(train_example["X"])
y = np.asarray(train_example["y"])
adj_mat = np.asarray(train_example["adjacency_matrix"])
num_nodes = train_example["num_nodes"]
density = train_example["density"]
mb = np.asarray(train_example["feature_mask"])

## 2. Visualize True DAG and Adjacency Matrix

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 7), constrained_layout=True)

plot_graph(adj_mat, figsize=(5, 5), ax=axes[0], title="True DAG")

plot_adjmat(
    adj_mat,
    title="Adjacency Matrix",
    figsize=(5, 5),
    ax=axes[1],
)

plt.show()

## 3. PC-based MB Discovery

In [ ]:
pc_feature, pc_adjmat = pc_selector(X, y, alpha=0.05, ci_test="fisherz", variant="stable")
ges_feature, ges_adjmat = ges_selector(X, y, criterion="bic", method="scatter")
direct_lingam_feature, direct_lingam_adjmat = direct_lingam_selector(
    X, y, measure="pwling", thresh=0.3
)
notears_feature, notears_adjmat = notears_selector(X, y, lambda1=0.1, loss_type="l2")

In [ ]:
# Compute graph metrics (SHD, precision, recall, F1) and MB metrics for each discovered graph

cdmb_results = {
    "PC": (pc_feature, pc_adjmat),
    "GES": (ges_feature, ges_adjmat),
    "DirectLiNGAM": (direct_lingam_feature, direct_lingam_adjmat),
    "NOTEARS": (notears_feature, notears_adjmat),
}

rows = []
for name, results in cdmb_results.items():
    feature = results[0]
    adj = results[1]
    shd_val = shd(adj_mat, adj)
    precision, recall, f1 = adjacency_confusion(adj_mat, adj)

    mb_jaccard = jaccard_score(mb, feature)
    mb_size = int(np.sum(feature))
    mb_reduction = reduction_rate(feature)

    rows.append(
        {
            "Method": name,
            "SHD": int(shd_val),
            "Precision": float(precision),
            "Recall": float(recall),
            "F1 Score": float(f1),
            "MB Jaccard": float(mb_jaccard),
            "MB Size": mb_size,
            "Reduction Rate": float(mb_reduction),
        }
    )

# Create and display comparison table
comparison_df = pd.DataFrame(rows)
comparison_df.sort_values(by="F1 Score", ascending=False, inplace=False).reset_index(drop=True)


In [ ]:
# visualize the results

num_methods = len(cdmb_results)
fig, axes = plt.subplots(2, num_methods + 1, figsize=(num_methods * 8, 12), constrained_layout=True)

plot_graph(
    adj_mat,
    figsize=(5, 5),
    title="True DAG",
    ax=axes[0, 0],
)

plot_adjmat(
    adj_mat,
    title="True Adjacency Matrix",
    figsize=(5, 5),
    ax=axes[1, 0],
)

for i, (name, results) in enumerate(cdmb_results.items(), 1):
    adj = results[1]
    plot_graph(
        adj,
        figsize=(5, 5),
        title=f"{name}",
        ax=axes[0, i],
    )

    plot_adjmat(
        adj,
        title=f"{name}",
        figsize=(5, 5),
        ax=axes[1, i],
    )

plt.show()

## 5. Define Training Function and Prepare Data


explain three [feature selection methods](https://sebastianraschka.com/faq/docs/feature_sele_categories.html)
1. filter: select feature by some metric with a threshold (e.g., $corr(X_i, y) > 0.8$)
2. wrapper: Use a wrapper model to perform feature selection. CB-MS
3. embedded: feature selection and training are integrated together. L1 regularization

We focus on wrapper and embedded methods

They have quite different training receipe

For Wrapper method
1. use the wrapper model to find optimal feature set
2. train a model using the feature set

For embedded method
1. train a model with embedded method
2. find optimal feature set
3. train the model again with the new feature set

In [ ]:
# Train linear models with different feature selections
from sklearn.model_selection import train_test_split

print("\n=== Training Linear Models ===")

# Create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = (X_train, y_train)
test_data = (X_test, y_test)

results = {}

# 1. Full features
results["full"] = linear_regression(train_data, test_data, feature_mask=None)

# 2. CD-MB methods (PC, GES, DirectLiNGAM, NOTEARS)
results["PC"] = linear_regression(train_data, test_data, feature_mask=pc_feature)
results["GES"] = linear_regression(train_data, test_data, feature_mask=ges_feature)
results["DirectLiNGAM"] = linear_regression(train_data, test_data, feature_mask=direct_lingam_feature)
results["NOTEARS"] = linear_regression(train_data, test_data, feature_mask=notears_feature)

# 3. L1 feature selection
l1_result = linear_l1_regression(train_data, test_data, alpha=0.05)
results["L1"] = l1_result
l1_feature = l1_result["feature_mask"]

# 4. Oracle (true Markov Blanket)
results["Oracle"] = linear_regression(train_data, test_data, feature_mask=mb)

print(f"Trained {len(results)} models")


In [ ]:
# Display results sorted by RMSE

df_results = pd.DataFrame({
    'Model': list(results.keys()),
    'RMSE': [results[k]['rmse'] for k in results.keys()],
    'MAE': [results[k]['mae'] for k in results.keys()],
    'R2': [results[k]['r2'] for k in results.keys()],
    'Features': [results[k]['n_features'] for k in results.keys()]
})

df_results.sort_values('RMSE', ascending=True).reset_index(drop=True)


In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. R2 comparison
ax1 = axes[0, 0]
df_results.plot(x='Model', y='R2', kind='bar', ax=ax1, color='skyblue', legend=False)
ax1.set_ylabel('R2 Score', fontweight='bold')
ax1.set_title('R2 Score (Higher is Better)', fontweight='bold')
ax1.set_xlabel('')
ax1.tick_params(axis='x', rotation=45)
ax1.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# 2. RMSE comparison
ax2 = axes[0, 1]
df_results.plot(x='Model', y='RMSE', kind='bar', ax=ax2, color='coral', legend=False)
ax2.set_ylabel('RMSE', fontweight='bold')
ax2.set_title('Root Mean Squared Error', fontweight='bold')
ax2.set_xlabel('')
ax2.tick_params(axis='x', rotation=45)

# 3. Predictions vs Ground Truth (Oracle MB)
ax3 = axes[1, 0]
y_pred_oracle = results['Oracle']['predictions']
y_test_oracle = results['Oracle']['truths']
ax3.scatter(y_test_oracle, y_pred_oracle, alpha=0.6, s=30)
ax3.plot([y_test_oracle.min(), y_test_oracle.max()], [y_test_oracle.min(), y_test_oracle.max()],
            'r--', lw=2, label='Perfect prediction')
ax3.set_xlabel('True Values', fontweight='bold')
ax3.set_ylabel('Predicted Values', fontweight='bold')
ax3.set_title('Oracle MB: Predictions vs Ground Truth', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Feature count vs R2 Performance
ax4 = axes[1, 1]
ax4.scatter(df_results['Features'], df_results['R2'], s=100, alpha=0.7)
for idx, row in df_results.iterrows():
    ax4.annotate(row['Model'], (row['Features'], row['R2']),
                xytext=(5, 5), textcoords='offset points', fontsize=9)
ax4.set_xlabel('Number of Features', fontweight='bold')
ax4.set_ylabel('R2', fontweight='bold')
ax4.set_title('Feature Efficiency', fontweight='bold')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Your Turn

From the above, we can see that Oracle MB gives the best prediction result on IID linear setting. However, it may just be a single case.

#### 1. Sample Size Analysis
**Objective**: Investigate how training data size affects feature selection methods.

- Vary the training set sizes (e.g., 50, 100, 500, 800)
- Plot RMSE vs. training sample size for each method
- Is Orcale MB robust in differernt sizes?
- Are CD-MB method robust on varing sizes?

#### 2. Nonlinear Relationships (Advanced)
**Objective**: Test methods on nonlinear datasets.

- Load the `'nonlinear'` subset: `load_dataset(path="CSE472-blanket-challenge/phase1-dataset", split='train', name='nonlinear')`
- Evaluate MLP models with feature selection
- Compare performance under nonlinear setting

## Bonus Points [5 points]

### Choose one:

#### 1. Graph Structure Analysis  
**Objective**: Understand how graph properties influence method performance.

- Test across graphs with varying:
  - **Density**:
  - **Markov Blanket size**: MB size w.r.t total feature size
  - **Graph size**: number of nodes

#### 2. L1 Regularization Tuning
**Objective**: Improve L1 feature selection through better hyperparameter selection.

- Implement adaptive alpha selection:
  - Cross-validation grid search
  - Use validation set to choose optimal alpha
  - Information criterion (AIC/BIC) based selection
- Compare with fixed alpha = 0.05
- **Key Question**: Can dynamic alpha selection make L1 competitive with Oracle MB?

## 6. Sample Size Analysis

Investigate how training data size affects feature selection methods.

In [ ]:
# Sample size analysis
from tqdm.auto import tqdm

train_sizes = [50, 100, 500, 800]
sample_size_results = []

# Use the same example dataset
print("Running sample size analysis...")

for size in tqdm(train_sizes, desc="Sample sizes"):
    # Subsample training data
    X_train_sub = X[:size]
    y_train_sub = y[:size]
    X_test_sub = X[size:size+200] if size+200 <= len(X) else X[size:]
    y_test_sub = y[size:size+200] if size+200 <= len(y) else y[size:]
    
    train_data_sub = (X_train_sub, y_train_sub)
    test_data_sub = (X_test_sub, y_test_sub)
    
    # Recompute feature selectors on subset
    pc_feat_sub, _ = pc_selector(X_train_sub, y_train_sub, alpha=0.05, ci_test="fisherz", variant="stable")
    ges_feat_sub, _ = ges_selector(X_train_sub, y_train_sub, criterion="bic", method="scatter")
    lingam_feat_sub, _ = direct_lingam_selector(X_train_sub, y_train_sub, measure="pwling", thresh=0.3)
    notears_feat_sub, _ = notears_selector(X_train_sub, y_train_sub, lambda1=0.1, loss_type="l2")
    
    # Test different methods
    methods = {
        'Full': None,
        'Oracle MB': mb,
        'PC': pc_feat_sub,
        'GES': ges_feat_sub,
        'DirectLiNGAM': lingam_feat_sub,
        'NOTEARS': notears_feat_sub,
    }
    
    # L1 method
    l1_res = linear_l1_regression(train_data_sub, test_data_sub, alpha=0.05)
    methods['L1'] = l1_res['feature_mask']
    
    for method_name, feature_mask in methods.items():
        result = linear_regression(train_data_sub, test_data_sub, feature_mask=feature_mask)
        sample_size_results.append({
            'Train Size': size,
            'Method': method_name,
            'RMSE': result['rmse'],
            'MAE': result['mae'],
            'R2': result['r2'],
            'N Features': result['n_features']
        })

sample_size_df = pd.DataFrame(sample_size_results)
print("Sample size analysis complete!")

In [ ]:
# Visualize sample size analysis
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Plot RMSE vs sample size
for method in sample_size_df['Method'].unique():
    method_data = sample_size_df[sample_size_df['Method'] == method]
    axes[0].plot(method_data['Train Size'], method_data['RMSE'], marker='o', label=method, linewidth=2)

axes[0].set_xlabel('Training Sample Size', fontweight='bold')
axes[0].set_ylabel('RMSE', fontweight='bold')
axes[0].set_title('RMSE vs Training Sample Size', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot R2 vs sample size
for method in sample_size_df['Method'].unique():
    method_data = sample_size_df[sample_size_df['Method'] == method]
    axes[1].plot(method_data['Train Size'], method_data['R2'], marker='o', label=method, linewidth=2)

axes[1].set_xlabel('Training Sample Size', fontweight='bold')
axes[1].set_ylabel('R² Score', fontweight='bold')
axes[1].set_title('R² vs Training Sample Size', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Display summary table
print("\n" + "="*80)
print("SAMPLE SIZE ANALYSIS SUMMARY")
print("="*80)
summary = sample_size_df.groupby(['Method', 'Train Size'])['RMSE'].mean().unstack()
print(summary)

In [ ]:
# Load nonlinear dataset
from blanket.models import mlp_regression

train_nonlinear = load_dataset(path="CSE472-blanket-challenge/phase1-dataset", split='train', name='nonlinear')

# Select same index for fair comparison
nonlinear_example = train_nonlinear[70]

X_nl = np.asarray(nonlinear_example["X"])
y_nl = np.asarray(nonlinear_example["y"])
adj_mat_nl = np.asarray(nonlinear_example["adjacency_matrix"])
mb_nl = np.asarray(nonlinear_example["feature_mask"])

print(f"Nonlinear dataset loaded: {X_nl.shape}")

# Create train/test split
X_train_nl, X_test_nl, y_train_nl, y_test_nl = train_test_split(X_nl, y_nl, test_size=0.2, random_state=42)
train_data_nl = (X_train_nl, y_train_nl)
test_data_nl = (X_test_nl, y_test_nl)

# Compute feature selectors on nonlinear data
print("Computing feature selectors...")
pc_feat_nl, _ = pc_selector(X_train_nl, y_train_nl, alpha=0.05, ci_test="fisherz", variant="stable")
ges_feat_nl, _ = ges_selector(X_train_nl, y_train_nl, criterion="bic", method="scatter")
lingam_feat_nl, _ = direct_lingam_selector(X_train_nl, y_train_nl, measure="pwling", thresh=0.3)
notears_feat_nl, _ = notears_selector(X_train_nl, y_train_nl, lambda1=0.1, loss_type="l2")
l1_res_nl = linear_l1_regression(train_data_nl, test_data_nl, alpha=0.05)

print("Feature selection complete!")

In [ ]:
# Compare Linear vs MLP performance on nonlinear data
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Comparison: Linear (from earlier) vs MLP (nonlinear data)
comparison = pd.DataFrame({
    'Method': df_results_nl['Model'],
    'MLP (Nonlinear)': df_results_nl['RMSE'],
    'Linear (From Section 5)': df_results['RMSE']  # From linear results earlier
})

comparison.plot(x='Method', kind='bar', ax=axes[0], rot=45)
axes[0].set_ylabel('RMSE', fontweight='bold')
axes[0].set_title('RMSE: MLP vs Linear Models', fontweight='bold')
axes[0].set_xlabel('')
axes[0].legend(['MLP (Nonlinear Data)', 'Linear (Linear Data)'])
axes[0].grid(True, alpha=0.3)

# R2 comparison
comparison_r2 = pd.DataFrame({
    'Method': df_results_nl['Model'],
    'MLP (Nonlinear)': df_results_nl['R2'],
    'Linear (From Section 5)': df_results['R2']
})

comparison_r2.plot(x='Method', kind='bar', ax=axes[1], rot=45)
axes[1].set_ylabel('R² Score', fontweight='bold')
axes[1].set_title('R²: MLP vs Linear Models', fontweight='bold')
axes[1].set_xlabel('')
axes[1].legend(['MLP (Nonlinear Data)', 'Linear (Linear Data)'])
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Train MLP models with different feature selections
results_nl = {}

print("\n=== Training MLP Models on Nonlinear Data ===")

# 1. Full features
results_nl["Full"] = mlp_regression(train_data_nl, test_data_nl, feature_mask=None)

# 2. CD-MB methods
results_nl["PC"] = mlp_regression(train_data_nl, test_data_nl, feature_mask=pc_feat_nl)
results_nl["GES"] = mlp_regression(train_data_nl, test_data_nl, feature_mask=ges_feat_nl)
results_nl["DirectLiNGAM"] = mlp_regression(train_data_nl, test_data_nl, feature_mask=lingam_feat_nl)
results_nl["NOTEARS"] = mlp_regression(train_data_nl, test_data_nl, feature_mask=notears_feat_nl)

# 3. L1
results_nl["L1"] = mlp_regression(train_data_nl, test_data_nl, feature_mask=l1_res_nl['feature_mask'])

# 4. Oracle MB
results_nl["Oracle MB"] = mlp_regression(train_data_nl, test_data_nl, feature_mask=mb_nl)

# Create results dataframe
df_results_nl = pd.DataFrame({
    'Model': list(results_nl.keys()),
    'RMSE': [results_nl[k]['rmse'] for k in results_nl.keys()],
    'MAE': [results_nl[k]['mae'] for k in results_nl.keys()],
    'R2': [results_nl[k]['r2'] for k in results_nl.keys()],
    'Features': [results_nl[k]['n_features'] for k in results_nl.keys()]
})

print(f"\nTrained {len(results_nl)} MLP models")
df_results_nl.sort_values('RMSE', ascending=True).reset_index(drop=True)

## 7. Nonlinear Relationships

Test feature selection methods on nonlinear datasets using MLP models.